In [ ]:
!pip install seaborn -q

In [1]:
!pip install datasets -q

In [1]:
import os
print(os.getpid())

2257117


In [2]:
import pandas as pd
import numpy as np

LABEL = 'mistake_identification'
MODEL = "roberta-large"
train_df = pd.read_csv("data/mistake_identification_train.csv")
val_df = pd.read_csv("data/mistake_identification_val.csv")

full_df = pd.concat([train_df, val_df]).reset_index(drop=True)

In [3]:
import torch, random, numpy as np

SEED = 42

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(SEED)

In [4]:
from collections import Counter

def get_weights(train_df, label, normalized=False):
  label_counts = Counter(train_df[label])
  total = sum(label_counts.values())
  weights = torch.tensor([total / label_counts.get(cls, 1) for cls in range(3)])
  if normalized:
    weights_normalized = weights / weights.mean()
    return weights_normalized
  return weights

In [5]:
alpha_equal = torch.tensor([1.0, 1.0, 1.0])
weights = get_weights(train_df, LABEL)
weights_normalized = get_weights(train_df, LABEL, True)

In [6]:
print(alpha_equal)
print(weights)
print(weights_normalized)

tensor([1., 1., 1.])
tensor([ 6.6768, 13.8671,  1.2852])
tensor([0.9176, 1.9058, 0.1766])


In [7]:
import torch.nn.functional as F
import torch.nn as nn

class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        logpt = F.log_softmax(inputs, dim=-1)
        pt = torch.exp(logpt)
        logpt = logpt.gather(1, targets.unsqueeze(1)).squeeze(1)
        pt = pt.gather(1, targets.unsqueeze(1)).squeeze(1)

        if self.alpha is not None:
            at = self.alpha.to(targets.device).gather(0, targets)
            logpt = logpt * at

        loss = -1 * (1 - pt) ** self.gamma * logpt

        if self.reduction == "mean":
            return loss.mean()
        elif self.reduction == "sum":
            return loss.sum()
        return loss

In [8]:
from torch.utils.data import Sampler
import numpy as np
from collections import defaultdict
import random

class StratifiedBatchSampler(Sampler):
    def __init__(self, labels, batch_size, shuffle=True):
        self.labels = np.array(labels)
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.label_to_indices = defaultdict(list)

        for idx, label in enumerate(self.labels):
            self.label_to_indices[label].append(idx)

    def __iter__(self):
        all_batches = []

        # Stratify each batch
        min_class_count = min([len(v) for v in self.label_to_indices.values()])
        n_batches = len(self.labels) // self.batch_size

        # Estimate proportions
        proportions = {
            k: len(v) / len(self.labels)
            for k, v in self.label_to_indices.items()
        }

        for _ in range(n_batches):
            batch = []
            for cls, prob in proportions.items():
                n_cls = max(1, int(prob * self.batch_size))
                chosen = random.sample(self.label_to_indices[cls], min(n_cls, len(self.label_to_indices[cls])))
                batch.extend(chosen)
            if self.shuffle:
                random.shuffle(batch)
            all_batches.append(batch[:self.batch_size])

        random.shuffle(all_batches)
        return iter([i for batch in all_batches for i in batch])

    def __len__(self):
        return len(self.labels)


In [9]:
from transformers import Trainer

class CustomTrainer(Trainer):
    def __init__(self, *args, loss_type="cross_entropy", **kwargs):
        super().__init__(*args, **kwargs)
        self.loss_type = loss_type
        self.weights = weights
        self.focal_loss = FocalLoss(alpha=alpha if loss_type == "focal" else None)

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
      labels = inputs.pop("labels")
      outputs = model(**inputs)
      logits = outputs.logits

      if self.loss_type == "focal":
          loss = self.focal_loss(logits, labels)
      elif self.loss_type == "weighted":
          w = self.weights.to(logits.device)
          loss = F.cross_entropy(logits, labels, weight=w)
      else:
          loss = F.cross_entropy(logits, labels)

      return (loss, outputs) if return_outputs else loss

    # Allows stratified batches - comment out if you don t want stratified batches
    # def get_train_dataloader(self):
    #     if self.train_dataset is None:
    #         raise ValueError("Trainer: training requires a train_dataset.")

    #     # Extract labels from dataset
    #     labels = self.train_dataset['labels']

    #     sampler = StratifiedBatchSampler(
    #         labels=labels,
    #         batch_size=self.args.train_batch_size,
    #         shuffle=True
    #     )

    #     return DataLoader(
    #         self.train_dataset,
    #         batch_size=self.args.train_batch_size,
    #         sampler=sampler,
    #         collate_fn=self.data_collator,
    #         drop_last=self.args.dataloader_drop_last,
    #         num_workers=self.args.dataloader_num_workers,
    #     )


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
TODO: adjust max_length based on 95th percentile for each case

TODO: prompt engineering

SyntaxError: invalid decimal literal (1456326944.py, line 1)

In [13]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained(MODEL)
def tokenize(batch, use_context=True):
    if use_context:
        text1 = [f"Student: {x}" for x in batch['last_student_turn']]
        text2 = [f"Tutor: {x}" for x in batch['response']]
        return tokenizer(text1, text2, truncation=True, padding="max_length", max_length=512)
    else:
        text = [x for x in batch['response']]
        return tokenizer(text, truncation=True, padding="max_length", max_length=512)

from functools import partial
tokenize_with_context = partial(tokenize, use_context=True)
tokenize_no_context = partial(tokenize, use_context=False)

In [14]:
from datasets import Dataset

train_ds_context = Dataset.from_pandas(train_df[["last_student_turn", "response", "mistake_identification"]].rename(columns={"mistake_identification": "labels"}))
val_ds_context = Dataset.from_pandas(val_df[["last_student_turn", "response", "mistake_identification"]].rename(columns={"mistake_identification": "labels"}))

train_ds_context = train_ds_context.map(tokenize_with_context, batched=True)
val_ds_context = val_ds_context.map(tokenize_with_context, batched=True)

train_ds_nocontext = Dataset.from_pandas(train_df[["last_student_turn", "response", "mistake_identification"]].rename(columns={"mistake_identification": "labels"}))
val_ds_nocontext = Dataset.from_pandas(val_df[["last_student_turn", "response", "mistake_identification"]].rename(columns={"mistake_identification": "labels"}))

train_ds_nocontext = train_ds_nocontext.map(tokenize_no_context, batched=True)
val_ds_nocontext = val_ds_nocontext.map(tokenize_no_context, batched=True)

Map: 100% 1983/1983 [00:00<00:00, 4912.84 examples/s]
Map: 100% 493/493 [00:00<00:00, 4655.66 examples/s]
Map: 100% 1983/1983 [00:00<00:00, 6145.22 examples/s]
Map: 100% 493/493 [00:00<00:00, 6048.74 examples/s]


In [17]:
from sklearn.metrics import f1_score, balanced_accuracy_score, classification_report, confusion_matrix
# Metrics
def compute_lenient_metrics(y_true, y_pred):
    def to_lenient(y): return np.where(np.isin(y, [1, 2]), 1, 0)
    y_true_len = to_lenient(y_true)
    y_pred_len = to_lenient(y_pred)

    acc = balanced_accuracy_score(y_true_len, y_pred_len)
    f1 = f1_score(y_true_len, y_pred_len, average="binary")
    return acc, f1


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    macro_f1 = f1_score(labels, preds, average="macro")
    acc = balanced_accuracy_score(labels, preds)
    len_acc, len_f1 = compute_lenient_metrics(labels, preds)
    conf = confusion_matrix(labels, preds)

    print("Confusion Matrix:")
    print(conf)
    # print(classification_report(labels, preds, target_names=["No", "TSE", "Yes"]))


    return {
        "accuracy": acc,
        "macro_f1": macro_f1,
        "lenient_accuracy": len_acc,
        "lenient_f1": len_f1
    }

In [18]:
from transformers import RobertaForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback

# Model
model = RobertaForSequenceClassification.from_pretrained(MODEL, num_labels=3)

# Training
args = TrainingArguments(
    output_dir="./outputs_mistake_id_roberta",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,                     # recommended for RoBERTa-large
    per_device_train_batch_size=8,          # reduce if OOM
    per_device_eval_batch_size=8,
    num_train_epochs=4,                     
    warmup_ratio=0.1,                       # important for RoBERTa stability
    weight_decay=0.01,
    logging_dir="./logs_roberta",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
    save_total_limit=2,
    lr_scheduler_type="linear",             # linear or cosine both work well
    fp16=torch.cuda.is_available(),         
    gradient_checkpointing=True,            # helps reduce memory
    report_to="none",                       # or "tensorboard"/"wandb" if needed
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
import torch, numpy as np, random
from transformers import RobertaForSequenceClassification
from collections import defaultdict
import pandas as pd

results = []

seeds = [42, 52, 62, 88, 99]
contexts = [True, False]

for use_context in contexts:
    for seed in seeds:
        print(f"\n==== Running: context={use_context}, seed={seed} ====")

        # Fix seed
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        np.random.seed(seed)
        random.seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

        # Tokenize
        train_ds = train_ds_context if use_context else train_ds_nocontext
        val_ds = val_ds_context if use_context else val_ds_nocontext

        # Define trainer
        model = RobertaForSequenceClassification.from_pretrained(MODEL, num_labels=3)

        trainer = CustomTrainer(
            model=model,
            args=args,
            train_dataset=train_ds,
            eval_dataset=val_ds,
            compute_metrics=compute_metrics,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
        )

        trainer.train()
        metrics = trainer.evaluate()

        # Save results
        results.append({
        "context": use_context,
        "seed": seed,
        "loss": metrics.get("eval_loss", -1),
        "macro_f1": metrics.get("eval_macro_f1", -1),
        "lenient_f1": metrics.get("eval_lenient_f1", -1),
        "accuracy": metrics.get("eval_accuracy", -1),
        "lenient_accuracy": metrics.get("eval_lenient_accuracy", -1)
      })


# Create result DataFrame
df_context = pd.DataFrame(results)
df_context.to_csv("results_context_comparison.csv", index=False)
print("\n All experiments complete. Results saved to 'results_context_comparison.csv'.")




==== Running: context=True, seed=42 ====


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [19]:
df_context

context  seed      loss  macro_f1  lenient_f1  accuracy  lenient_accuracy
0     True    42  0.555945  0.700287    0.965842  0.894523          0.941176
1     True    52  0.536447  0.687964    0.963572  0.888438          0.937120
2     True    62  0.593741  0.675755    0.962175  0.886410          0.935091
3     True    88  0.558683  0.673045    0.962264  0.882353          0.935091
4     True    99  0.520346  0.676742    0.965842  0.894523          0.941176
5    False    42  0.532795  0.629411    0.961039  0.876268          0.933063
6    False    52  0.521937  0.686720    0.962264  0.886410          0.935091
7    False    62  0.489638  0.649745    0.963400  0.882353          0.937120
8    False    88  0.546296  0.629348    0.956419  0.868154          0.924949
9    False    99  0.515535  0.730862    0.965922  0.888438          0.941176

In [23]:
import seaborn as sns
import matplotlib.pyplot as plt

df_context = pd.read_csv("results_context_comparison.csv")
sns.boxplot(data=df_context, x="context", y="macro_f1")
plt.title("Macro F1 without vs with context")
plt.show()

In [ ]:
Roberta large performs better with context, across all metrics and seeds. E un outlier in care pt no context am obt 0.73.

In [28]:
alpha = alpha_equal
alpha

tensor([1., 1., 1.])

In [25]:
weights = weights_normalized
weights

tensor([0.9176, 1.9058, 0.1766])

In [ ]:
*Din punctul asta am folosit balanced_accuracy_score 

In [31]:
import torch, numpy as np, random
from collections import defaultdict
import pandas as pd

results = []

seeds = [42, 52, 62, 88, 99]
loss_types = ["weighted", "focal"]

for loss_type in loss_types:
    for seed in seeds:
        print(f"\n==== Running: loss_type={loss_type}, seed={seed} ====")

        # Fix seed
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        np.random.seed(seed)
        random.seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

        # Define trainer
        model = RobertaForSequenceClassification.from_pretrained(MODEL, num_labels=3)

        trainer = CustomTrainer(
            model=model,
            args=args,
            train_dataset=train_ds_context, # ALWAYS USING CONTEXT FROM NOW ON
            eval_dataset=val_ds_context,
            compute_metrics=compute_metrics,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
            loss_type=loss_type
        )

        trainer.train()
        metrics = trainer.evaluate()

        # Save results
        results.append({
        "loss_type": loss_type,
        "seed": seed,
        "loss": metrics.get("eval_loss", -1),
        "macro_f1": metrics.get("eval_macro_f1", -1),
        "lenient_f1": metrics.get("eval_lenient_f1", -1),
        "accuracy": metrics.get("eval_accuracy", -1),
        "lenient_accuracy": metrics.get("eval_lenient_accuracy", -1)
      })


# Create result DataFrame
df_loss = pd.DataFrame(results)
df_loss.to_csv("results_loss_comparison.csv", index=False)


==== Running: loss_type=weighted, seed=42 ====


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Confusion Matrix:
[[ 36  12  25]
 [  1  12  18]
 [  0  32 357]]
              precision    recall  f1-score   support

          No       0.97      0.49      0.65        73
         TSE       0.21      0.39      0.28        31
         Yes       0.89      0.92      0.90       389

    accuracy                           0.82       493
   macro avg       0.69      0.60      0.61       493
weighted avg       0.86      0.82      0.83       493

Confusion Matrix:
[[ 50   3  20]
 [  2  11  18]
 [ 16  12 361]]
              precision    recall  f1-score   support

          No       0.74      0.68      0.71        73
         TSE       0.42      0.35      0.39        31
         Yes       0.90      0.93      0.92       389

    accuracy                           0.86       493
   macro avg       0.69      0.66      0.67       493
weighted avg       0.85      0.86      0.85       493

Confusion Matrix:
[[ 50   1  22]
 [  4   6  21]
 [ 13   5 371]]
              precision    recall  f1-score   

Confusion Matrix:
[[ 50   3  20]
 [  2  11  18]
 [ 16  12 361]]
              precision    recall  f1-score   support

          No       0.74      0.68      0.71        73
         TSE       0.42      0.35      0.39        31
         Yes       0.90      0.93      0.92       389

    accuracy                           0.86       493
   macro avg       0.69      0.66      0.67       493
weighted avg       0.85      0.86      0.85       493


==== Running: loss_type=weighted, seed=52 ====


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Confusion Matrix:
[[ 43   0  30]
 [  0   0  31]
 [  1   0 388]]
              precision    recall  f1-score   support

          No       0.98      0.59      0.74        73
         TSE       0.00      0.00      0.00        31
         Yes       0.86      1.00      0.93       389

    accuracy                           0.87       493
   macro avg       0.61      0.53      0.55       493
weighted avg       0.83      0.87      0.84       493



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Confusion Matrix:
[[ 46   2  25]
 [  0   8  23]
 [  3   7 379]]
              precision    recall  f1-score   support

          No       0.94      0.63      0.75        73
         TSE       0.47      0.26      0.33        31
         Yes       0.89      0.97      0.93       389

    accuracy                           0.88       493
   macro avg       0.77      0.62      0.67       493
weighted avg       0.87      0.88      0.87       493

Confusion Matrix:
[[ 53   2  18]
 [  3   6  22]
 [ 13   5 371]]
              precision    recall  f1-score   support

          No       0.77      0.73      0.75        73
         TSE       0.46      0.19      0.27        31
         Yes       0.90      0.95      0.93       389

    accuracy                           0.87       493
   macro avg       0.71      0.62      0.65       493
weighted avg       0.86      0.87      0.86       493

Confusion Matrix:
[[ 52   2  19]
 [  2   9  20]
 [  9   5 375]]
              precision    recall  f1-score   

Confusion Matrix:
[[ 52   2  19]
 [  2   9  20]
 [  9   5 375]]
              precision    recall  f1-score   support

          No       0.83      0.71      0.76        73
         TSE       0.56      0.29      0.38        31
         Yes       0.91      0.96      0.93       389

    accuracy                           0.88       493
   macro avg       0.76      0.66      0.69       493
weighted avg       0.87      0.88      0.87       493



In [32]:
df_loss

loss_type  seed      loss  macro_f1  lenient_f1  accuracy  lenient_accuracy
0  weighted    42  0.812704  0.670476    0.951479  0.655930          0.821037
1  weighted    52  1.159253  0.674565    0.965922  0.639472          0.852299
2  weighted    62  1.062273  0.672352    0.949275  0.669241          0.865117
3  weighted    88  0.844411  0.689547    0.963743  0.662785          0.832942
4  weighted    99  0.846857  0.679166    0.954976  0.689609          0.835926
5     focal    42  0.241446  0.691545    0.964871  0.644519          0.839791
6     focal    52  0.246164  0.675118    0.963572  0.636619          0.844260
7     focal    62  0.255305  0.689710    0.964706  0.650224          0.851109
8     focal    88  0.227770  0.686596    0.965438  0.627593          0.800179
9     focal    99  0.243520  0.693894    0.962353  0.655554          0.843069

In [38]:
# Filter only context == False rows
df_ce = df_context[df_context["context"] == True].copy()
df_ce["loss_type"] = "ce"
df_ce = df_ce[["loss_type", "seed", "loss", "macro_f1", "lenient_f1", "accuracy", "lenient_accuracy"]]
df_loss = pd.concat([df_loss, df_ce], ignore_index=True)
df_loss

loss_type  seed      loss  macro_f1  lenient_f1  accuracy  lenient_accuracy
0   weighted    42  0.812704  0.670476    0.951479  0.655930          0.821037
1   weighted    52  1.159253  0.674565    0.965922  0.639472          0.852299
2   weighted    62  1.062273  0.672352    0.949275  0.669241          0.865117
3   weighted    88  0.844411  0.689547    0.963743  0.662785          0.832942
4   weighted    99  0.846857  0.679166    0.954976  0.689609          0.835926
5      focal    42  0.241446  0.691545    0.964871  0.644519          0.839791
6      focal    52  0.246164  0.675118    0.963572  0.636619          0.844260
7      focal    62  0.255305  0.689710    0.964706  0.650224          0.851109
8      focal    88  0.227770  0.686596    0.965438  0.627593          0.800179
9      focal    99  0.243520  0.693894    0.962353  0.655554          0.843069
10        ce    42  0.532795  0.629411    0.961039  0.876268          0.933063
11        ce    52  0.521937  0.686720    0.962264  0.886410          0.935091
12        ce    62  0.489638  0.649745    0.963400  0.882353          0.937120
13        ce    88  0.546296  0.629348    0.956419  0.868154          0.924949
14        ce    99  0.515535  0.730862    0.965922  0.888438          0.941176

In [40]:
plt.figure(figsize=(8, 6))
sns.boxplot(data=df_loss, x="loss_type", y="macro_f1")
plt.title("Macro F1 Score Across Loss Functions")
plt.ylabel("Macro F1")
plt.xlabel("Loss Function")
plt.grid(True)
plt.tight_layout()
plt.show()

Summary:
Context is the best for roberta-large.
Focal loss is the best for roberta-large.

In [41]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from math import log
from collections import Counter
from transformers import AutoModelForSequenceClassification

samples_per_cls = [Counter(train_df[LABEL])[i] for i in range(3)]

class CBFocalLoss(nn.Module):
    """
    Class‑Balanced Focal Loss
        - beta controls how aggressively rare classes are up‑weighted
        - gamma is the focal‐loss focusing parameter
    """
    def __init__(self, samples_per_cls, beta=0.99, gamma=2.0, reduction='mean'):
        super().__init__()
        n_cls = len(samples_per_cls)

        effective_num = 1.0 - torch.pow(torch.tensor(beta), torch.tensor(samples_per_cls))
        self.weights = ((1.0 - beta) / effective_num).float()          
        self.weights = self.weights / self.weights.sum() * n_cls       # re‑normalise
        self.gamma   = gamma
        self.reduction = reduction

    def forward(self, logits, targets):
        logpt = F.log_softmax(logits, dim=-1)          # log‑prob for every class
        pt    = torch.exp(logpt)                       # prob for every class

        logpt = logpt.gather(1, targets.unsqueeze(1)).squeeze(1)
        pt    = pt.gather(1, targets.unsqueeze(1)).squeeze(1)

        alpha_t = self.weights.to(logits.device).gather(0, targets)
        loss = -alpha_t * (1 - pt) ** self.gamma * logpt

        return loss.mean() if self.reduction == 'mean' else loss.sum()


class CBTrainer(CustomTrainer):
    def __init__(self, *args, beta=0.99, gamma=2.0, **kwargs):
        super().__init__(*args, **kwargs)
        self.cb_loss = CBFocalLoss(samples_per_cls,
                                   beta=beta, gamma=gamma)

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits  = outputs.logits
        loss    = self.cb_loss(logits, labels)
        return (loss, outputs) if return_outputs else loss


beta_grid  = [0.9, 0.99]
gamma_grid = [1.0, 2.0]
seeds      = [42, 52, 62, 88, 99]

cb_results = []

for beta in beta_grid:
    for gamma in gamma_grid:
        for seed in seeds:
            print(f"\n=== CB‑Focal: beta={beta}, gamma={gamma}, seed={seed} ===")

            # reproducibility
            torch.manual_seed(seed)
            torch.cuda.manual_seed_all(seed)
            np.random.seed(seed)
            random.seed(seed)

            model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=3)

            trainer = CBTrainer(
                model=model,
                args=args,
                train_dataset=train_ds_nocontext,      # response‑only proved strongest
                eval_dataset=val_ds_nocontext,
                compute_metrics=compute_metrics,
                beta=beta,
                gamma=gamma,
                callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
            )

            trainer.train()
            metrics = trainer.evaluate()

            cb_results.append({
                "beta": beta,
                "gamma": gamma,
                "seed": seed,
                "loss": metrics.get("eval_loss", -1),
                "macro_f1": metrics.get("eval_macro_f1", -1),
                "lenient_f1": metrics.get("eval_lenient_f1", -1),
                "accuracy": metrics.get("eval_accuracy", -1),
                "lenient_accuracy": metrics.get("eval_lenient_accuracy", -1)
            })

df_cb = pd.DataFrame(cb_results)
df_cb.to_csv("results_cb_focal.csv", index=False)
display(df_cb.groupby(["beta", "gamma"]).agg({"macro_f1":"mean"}).sort_values("macro_f1", ascending=False))


=== CB‑Focal: beta=0.9, gamma=1.0, seed=42 ===


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Confusion Matrix:
[[ 45   1  27]
 [  2   4  25]
 [  4   1 384]]
              precision    recall  f1-score   support

          No       0.88      0.62      0.73        73
         TSE       0.67      0.13      0.22        31
         Yes       0.88      0.99      0.93       389

    accuracy                           0.88       493
   macro avg       0.81      0.58      0.62       493
weighted avg       0.87      0.88      0.86       493

Confusion Matrix:
[[ 47   2  24]
 [  1   5  25]
 [  2   2 385]]
              precision    recall  f1-score   support

          No       0.94      0.64      0.76        73
         TSE       0.56      0.16      0.25        31
         Yes       0.89      0.99      0.94       389

    accuracy                           0.89       493
   macro avg       0.79      0.60      0.65       493
weighted avg       0.87      0.89      0.87       493

Confusion Matrix:
[[ 53   1  19]
 [  2   4  25]
 [ 14   1 374]]
              precision    recall  f1-score   

Confusion Matrix:
[[ 53   1  19]
 [  2   6  23]
 [  9   4 376]]
              precision    recall  f1-score   support

          No       0.83      0.73      0.77        73
         TSE       0.55      0.19      0.29        31
         Yes       0.90      0.97      0.93       389

    accuracy                           0.88       493
   macro avg       0.76      0.63      0.66       493
weighted avg       0.87      0.88      0.87       493


=== CB‑Focal: beta=0.9, gamma=1.0, seed=52 ===


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Confusion Matrix:
[[ 52   2  19]
 [  3   8  20]
 [  8   4 377]]
              precision    recall  f1-score   support

          No       0.83      0.71      0.76        73
         TSE       0.57      0.26      0.36        31
         Yes       0.91      0.97      0.94       389

    accuracy                           0.89       493
   macro avg       0.77      0.65      0.69       493
weighted avg       0.87      0.89      0.87       493


=== CB‑Focal: beta=0.99, gamma=2.0, seed=99 ===


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Confusion Matrix:
[[ 41  10  22]
 [  3   8  20]
 [  4  18 367]]
              precision    recall  f1-score   support

          No       0.85      0.56      0.68        73
         TSE       0.22      0.26      0.24        31
         Yes       0.90      0.94      0.92       389

    accuracy                           0.84       493
   macro avg       0.66      0.59      0.61       493
weighted avg       0.85      0.84      0.84       493

Confusion Matrix:
[[ 50   2  21]
 [  3   7  21]
 [  8   2 379]]
              precision    recall  f1-score   support

          No       0.82      0.68      0.75        73
         TSE       0.64      0.23      0.33        31
         Yes       0.90      0.97      0.94       389

    accuracy                           0.88       493
   macro avg       0.79      0.63      0.67       493
weighted avg       0.87      0.88      0.87       493

Confusion Matrix:
[[ 50   1  22]
 [  2   4  25]
 [  6   1 382]]
              precision    recall  f1-score   

Confusion Matrix:
[[ 54   2  17]
 [  1   9  21]
 [ 11   6 372]]
              precision    recall  f1-score   support

          No       0.82      0.74      0.78        73
         TSE       0.53      0.29      0.38        31
         Yes       0.91      0.96      0.93       389

    accuracy                           0.88       493
   macro avg       0.75      0.66      0.69       493
weighted avg       0.87      0.88      0.87       493



macro_f1
beta gamma          
0.99 2.0    0.677060
     1.0    0.676957
0.90 2.0    0.675737
     1.0    0.667432

deci nu prea relevant